# Subtyping

Now that we understand that user-defined classes are just types that we can write from scratch to suit our use case, we might wonder how we can adapt existing classes to fit our needs. In this notebook, we are going to introduce the idea of inheritance.

To motivate this idea, we'll reuse our very simple `IncomeStatement` class, and extend the class to add complexity.

In [5]:
from typing import List, Dict, Callable
import pandas as pd

## Base Class

Our super simple `IncomeStatement` features only has basic components of an income statement, which should leave plenty of room to complicate matters. Don't worry too much about the methods beyond the constructor, which are really only in play to make it easy to visually inspect the statements.

In [20]:
class StatementPeriod:
    
    def __init__(self, start: pd.Timestamp, end: pd.Timestamp) -> None:
        self.start: pd.Timestamp = start
        self.end: pd.Timestamp = end
        self.duration: pd.Timedelta = self.end - self.start
            
    def __str__(self) -> str:
        return f"StatementPeriod(start={self.start}, end={self.end}, duration={self.duration})"
    
    def __repr__(self) -> str:
        return str(self)

class IncomeStatement:

    def __init__(
        self,
        period: StatementPeriod,
        revenues: float,
        cogs: float,
        tax_rate: float
    ) -> None:
        self.period: StatementPeriod = period
        self.revenues: float = revenues
        self.cogs: float = cogs
        self.tax_rate: float = tax_rate
        self.gross_margin: float = self.revenues - self.cogs
        self.tax: float = self.tax_rate * self.gross_margin
        self.earnings: float = self.gross_margin - self.tax
            
    def to_df(self) -> pd.DataFrame:
        out: pd.DataFrame = pd.DataFrame(self.__dict__, index=[self.period.end]).drop("period", axis=1)
        out["duration"] = self.period.duration
        out = out[["duration"] + [col for col in out.columns if col != "duration"]]
        return out.T    
            
    def __str__(self) -> str:
        return str(self.to_df())
    
    def __repr__(self) -> str:
        return f"IncomeStatement({str(self.__dict__)})"
    
    @staticmethod
    def many_to_df(stmts: List["IncomeStatement"]) -> pd.DataFrame:
        out: pd.DataFrame = pd.concat([
            inc.to_df() for inc in stmts
        ], axis=1)
        return out
    
    @staticmethod
    def many_to_str(stmts: List["IncomeStatement"]) -> str:
        df: pd.DataFrame = IncomeStatement.many_to_df(stmts)
        return str(df)
        

example_income_1: IncomeStatement = IncomeStatement(
    period=StatementPeriod(pd.Timestamp("2020-01-01"), pd.Timestamp("2020-12-31")),
    revenues=100.,
    cogs=80.,
    tax_rate=0.2
)
example_income_2: IncomeStatement = IncomeStatement(
    period=StatementPeriod(pd.Timestamp("2021-01-01"), pd.Timestamp("2021-12-31")),
    revenues=100.,
    cogs=80.,
    tax_rate=0.2
)
    
print(IncomeStatement.many_to_str(stmts=[example_income_1, example_income_2]))

                     2020-12-31         2021-12-31
duration      365 days 00:00:00  364 days 00:00:00
revenues                    100                100
cogs                         80                 80
tax_rate                    0.2                0.2
gross_margin                 20                 20
tax                           4                  4
earnings                     16                 16


Now suppose we want to add the capacity to include debt? Our class is already written, so what do we do now?

## [Inheritance](https://realpython.com/inheritance-composition-python/) (a.k.a. nominal subtyping)

Inheritance is about parent-child relationships between types. In this case, the parent tends to be the more general type, and the child tends to be the more specific. The classic example is to have some class `Animal` and another class `Dog` that inherits from it. The `Dog` type would then need to have the members (i.e. attributes and behaviors) that `Animal` has, while retaining the option to add members or alter their implementation. Sticking with the theme, our parent class will be `IncomeStatement` and our child class will be `DetailedIncomeStatement` which extends `IncomeStatement` with information about long-term debt.

Notice our implementation will differ in a couple different ways:

1. When we declare our class, we put the parent class (i.e. `IncomeStatement`) in parentheses after the class name. This is how we tell Python that we are inheriting from it.
2. We *only* implement the constructor because we are updating our attributes. We do absolutely nothing to implement the remaining methods.

In [24]:
class DetailedIncomeStatement(IncomeStatement):

    def __init__(
        self,
        period: StatementPeriod,
        revenues: float,
        cogs: float,
        debt_outstanding: float,
        interest_rate: float,
        tax_rate: float
    ) -> None:
        self.period: StatementPeriod = period
        self.revenues: float = revenues
        self.cogs: float = cogs
        self.debt_outstanding: float = debt_outstanding
        self.interest_rate: float = interest_rate
        self.debt_service: float = self.debt_outstanding * self.interest_rate
        self.tax_rate: float = tax_rate
        self.gross_margin: float = self.revenues - self.cogs
        self.pretax: float = self.gross_margin - self.debt_service
        self.tax: float = self.tax_rate * self.pretax
        self.earnings: float = self.pretax - self.tax
            
detailed_income_1: DetailedIncomeStatement = DetailedIncomeStatement(
    period=StatementPeriod(pd.Timestamp("2020-01-01"), pd.Timestamp("2020-12-31")),
    revenues=100.,
    cogs=80.,
    debt_outstanding=100.,
    interest_rate=0.05,
    tax_rate=0.2
)
detailed_income_2: DetailedIncomeStatement = DetailedIncomeStatement(
    period=StatementPeriod(pd.Timestamp("2021-01-01"), pd.Timestamp("2021-12-31")),
    revenues=100.,
    cogs=80.,
    debt_outstanding=100.,
    interest_rate=0.05,
    tax_rate=0.2
)

Hmmm, it would be nice to inspect our classes, but we didn't write new methods to represent the data in a human-friendly way. What happens if we try to print it?

In [26]:
print(str(detailed_income_1))

                         2020-12-31
duration          365 days 00:00:00
revenues                        100
cogs                             80
debt_outstanding                100
interest_rate                  0.05
debt_service                      5
tax_rate                        0.2
gross_margin                     20
pretax                           15
tax                               3
earnings                         12


Woah! Even though we didn't tell `DetailedIncomeStatement` how to print, it somehow inferred what we wanted to see. The key is that we inherited this way of printing the data from `IncomeStatement` via inheritance. We implemented the `__str__()` method for `IncomeStatement`, so that implementation still applies unless we explicitly decide to reimplement. This behavior is a key benefit of subtyping: *even as we complicate our ask we can leverage our previous investment to get some things for free*.

What about those static methods. They were written explicitly for instances of `IncomeStatement`, not `DetailedIncomeStatement`. Can we still use them?

In [27]:
print(IncomeStatement.many_to_str(stmts=[detailed_income_1, detailed_income_2]))

                         2020-12-31         2021-12-31
duration          365 days 00:00:00  364 days 00:00:00
revenues                        100                100
cogs                             80                 80
debt_outstanding                100                100
interest_rate                  0.05               0.05
debt_service                      5                  5
tax_rate                        0.2                0.2
gross_margin                     20                 20
pretax                           15                 15
tax                               3                  3
earnings                         12                 12


Enter the second major benefit: *if type B is a subtype of type A, we can use type B everywhere that we can use type A*. In our case, since `DetailedIncomeStatement` is a subtype of `IncomeStatement`, we can use `DetailedIncomeStatement` everywhere that we are allowed to use `IncomeStatement`. For you nerds out there, this is called the [Liskov Substitution Principle](https://en.wikipedia.org/wiki/Liskov_substitution_principle). It provides us with another source of flexibility by again allowing us to leverage previous investment.